In [1]:
%matplotlib inline
%config InlineBackend.figure_format='svg'
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
from numpy import pi,linspace,sin,exp,round,zeros,arange,real, tan
from numpy.fft import fft,ifft
from matplotlib.pyplot import figure
from time import time
from scipy.linalg import toeplitz

In [2]:
M = [128, 258, 512, 1024]
T = []
for N in M:
    ti1 = time()
    # Set up grid and differentiation matrix:
    h = 2*pi/N; x = h*arange(1,N+1);
    t = 0.0; dt = h/4.0
    c = 0.2 + sin(x-1.0)**2.0
    v = exp(-100.0*(x-1.0)**2.0); vold = exp(-100.0*(x-0.2*dt-1.0)**2.0);

    # Time-stepping by leap-frog formula
    tmax = 8.0; tplot = 0.15;
    plotgap = int(round(tplot/dt)); dt = tplot/plotgap;
    nplots = int(round(tmax/tplot))
    data = []
    data.append(list(zip(x, v)))
    tdata = zeros(nplots+1)
    tf1 = time()
    
    t1 = tf1 - ti1
    
    for j in range(2):
        ti2 = time()
        for i in range(1,nplots):
            for n in range(plotgap):
                t = t + dt
                if j==0:
                    v_hat = fft(v)
                    w_hat = 1j*zeros(N)
                    w_hat[0:N/2] = 1j*arange(0,N/2)
                    w_hat[N/2+1:] = 1j*arange(-N/2+1,0,1)
                    w_hat = w_hat * v_hat
                    w = real(ifft(w_hat)) 
                elif j==1:
                    col = zeros(N)
                    col[1:] = 0.5*(-1.0)**arange(1,N)/tan(arange(1,N)*h/2.0)
                    row = zeros(N)
                    row[0] = col[0]
                    row[1:] = col[N-1:0:-1]
                    D = toeplitz(col,row)    
                    w = D.dot(v)             # differentiation matrix:
                vnew = vold - 2.0*dt*c*w
                vold = v; v = vnew;
            data.append(list(zip(x, v)))
            tdata[i] = t;
        tf2 = time()
        t2 = tf2 - ti2
        t_exe = t1 + t2
        T.append(t_exe)

print 'Time_FFT =', T[0:len(T):2]
print 'Time_matrix =', T[1:len(T):2]



Time_FFT = [0.0384068489074707, 0.11110377311706543, 0.22235321998596191, 0.5486018657684326]
Time_matrix = [0.13942384719848633, 0.47411108016967773, 2.7295761108398438, 24.786754846572876]
